# Лабараторная работа 1

## Часть 1 - сбор данных

Суть - написать парсер текстовых данных (статей)
Нужно использовать 1 из предложенных вариантов (а также можно использовать свой пример)

Варианты сайтов для парсинга:
1. https://lenta.ru
2. https://naked-science.ru/
3. https://ngs24.ru/
4. https://goldenmost.ru/
5. https://nta-pfo.ru/

Можно использовать любой другой источник.

Необходимо сохранить следующие сущности
1. title - заголовок статьи
2. content - текст статьи
3. category - категория статьи
4. comments - (при наличии)
5. created_date - дата создания статьи (при наличии)
6. // прочие данные, которые вы посчитаете полезной)

Примеры парсеров расположены в lecture_1:
1. 1_sync_parser - самый простой и топорный вариант
2. 2_thread_parser - парсер основанный на многопоточности
3. 3_async_parser - парсер основанный на асинхронности
4. 4_browser_parser - парсер основанный на создании вирртуального парсера (полезно, когда сайт основан на client-side rendering)

P.S. Для запуска - необходимо установить python 3.11 (https://www.python.org/downloads/release/python-3110/)

Установить зависимости
```pip3 install -r requirements.txt```

Запуск файлов
``` python3 lecture_1/1_sync_parser/main.py ```


Для тех кто под *, необходимо использовать варианты 3,4 либо использовать фреймворки для парсинга, например **scrappy** (https://scrapy.org/)
P.S. запрещено использовать naked-science.ru если вы используете scrappy (пример кода [парсинг naked-science.ru](https://disk.yandex.ru/d/rsZ2sUvjj4lfUg))

Разрешенные форматы хранения результатов
- .csv
- .json
- .sqlite3

## Часть 2 - анализ текстовых данных

Провести аналитику текстовых данных.

1. Токенезировать и лемматизировать полученные текстовые данные полученные из 1 части задания

- Токенизация - разделение слов на предложения
- Лемматизация - перевод слов в начальную форму

2. Посчитать наиболее встречающиеся пары подлежащих и сказуемых
3. Посчитать самые популлярные слова (исключая стоп-слова, словарь стоп слов можно найти в nltk, либо отбросить по частям речи)
4. Вывести статистику (по убыванию, наиболее встречающиеся пары и самые популрняые стоп слоова)

In [22]:
data = [
    {'text': 'Эти типы стали есть в цеху. Пример токенизации. Для примера мы токенезируем'}, 
    {'text': 'Научный советник рассказал о... (читать продолжение в источнике).'}, 
    {'text': 'В статье говориться о котиках.'},
    {'text': 'Стекло бьется.'}
] # p.s. здесь вы открываете полученные данные

# Для примера можно использовать natasha
# !pip install natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [24]:
from ipymarkup import show_dep_ascii_markup as show_markup
for article in data:
    text = article.get('text')
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.parse_syntax(syntax_parser)
    doc.tag_morph(morph_tagger)
    for sentence in doc.sents: # получаем предложения
        sentence.syntax.print()
#         for token in sentence.tokens: # получаем токены
#             print(token)

      ┌► Эти   det
    ┌►└─ типы  nsubj
┌─┌─└─┌─ стали 
│ │   └► есть  xcomp
│ │   ┌► в     case
│ └──►└─ цеху  obl
└──────► .     punct
   Пример      
┌─ токенизации 
└► .           punct
  ┌► Для          case
┌►└─ примера      obl
│ ┌► мы           nsubj
└─└─ токенезируем 
    ┌► Научный   amod
  ┌►└─ советник  nsubj
┌─└─┌─ рассказал 
│   └► о         advmod
└────► ...       punct
        ┌► (           punct
┌─┌───┌─└─ читать      
│ │ ┌─└──► продолжение obj
│ │ │   ┌► в           case
│ │ └──►└─ источнике   nmod
│ └──────► )           punct
└────────► .           punct
      ┌► В          case
    ┌►└─ статье     obl
┌─┌─└─── говориться 
│ │   ┌► о          case
│ └──►└─ котиках    obl
└──────► .          punct
  ┌► Стекло obj
┌─└─ бьется 
└──► .      punct


In [25]:
# Продолжение следует...